# imports

In [5]:
import pandas as pd
import json
import os
import requests
import subprocess
import psycopg2
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display 


In [65]:
conn=psycopg2.connect(host="localhost", user="postgres", password="chicken65", database="phonepe")
cursor=conn.cursor()

# git cloning

In [ ]:
git.Repo.clone_from("https://github.com/PhonePe/pulse.git","phonepe_pulse")

# agg transaction

In [6]:
aggtranspath="D:/data science/projects/phonepe_pulse/data/aggregated/transaction/country/india/state"
agg_st_list=os.listdir(aggtranspath)
clm={"state":[],"year":[],"quarter":[],"transaction_type":[],"transaction_count":[],"transaction_amount":[]}
for i in agg_st_list:
  path_i=aggtranspath+"/"+i
  agg_yr=os.listdir(path_i)
  for j in agg_yr:
    path_j=path_i+"/"+j
    agg_yr_list=os.listdir(path_j)
    for k in agg_yr_list:
      path_k=path_j+"/"+k
      data=open(path_k,"r")
      d=json.load(data)
      for z in d["data"]["transactionData"]:
        name=z["name"]
        count=z["paymentInstruments"][0]["count"]
        amount=z["paymentInstruments"][0]["amount"]
        clm["transaction_type"].append(name)
        clm["transaction_count"].append(count)
        clm["transaction_amount"].append(amount)
        clm["state"].append(i)
        clm["year"].append(j)
        clm["quarter"].append(int(k.strip(".json")))
agg_trans=pd.DataFrame(clm)

agg_trans["state"]=agg_trans["state"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
agg_trans["state"]=agg_trans["state"].str.replace("---"," & ")
agg_trans["state"]=agg_trans["state"].str.replace("-"," ")
agg_trans["state"]=agg_trans["state"].str.replace("Dadra & Nagar Haveli & Daman & diu","Dadra and Nagar Haveli and Daman and Diu")
agg_trans["state"]=agg_trans["state"].str.title()

In [48]:
agg_trans


,state,year,quarter,transaction_type,transaction_count,transaction_amount
0,Andaman & Nicobar,2018,1,Recharge & bill payments,4200,1.845307e+06
1,Andaman & Nicobar,2018,1,Peer-to-peer payments,1871,1.213866e+07
2,Andaman & Nicobar,2018,1,Merchant payments,298,4.525072e+05
3,Andaman & Nicobar,2018,1,Financial Services,33,1.060142e+04
4,Andaman & Nicobar,2018,1,Others,256,1.846899e+05
...,...,...,...,...,...,...
3589,West Bengal,2022,4,Peer-to-peer payments,184380244,6.202222e+11
3590,West Bengal,2022,4,Merchant payments,171667404,1.408077e+11
3591,West Bengal,2022,4,Recharge & bill payments,48921147,2.602663e+10
3592,West Bengal,2022,4,Financial Services,268388,2.611229e+08


# agg user

In [49]:
agguserpath="D:/data science/projects/phonepe_pulse/data/aggregated/user/country/india/state"
agg_st_list1=os.listdir(agguserpath)
clm1={"state":[],"year":[],"quarter":[],"brands":[],"count":[],"percentage":[]}
for i in agg_st_list1:
  path_i=agguserpath+"/"+i
  agg_yr=os.listdir(path_i)
  for j in agg_yr:
    path_j=path_i+"/"+j
    agg_yr_list=os.listdir(path_j)
    for k in agg_yr_list:
      path_k=os.path.join(path_j,k)
      if os.path.isfile(path_k):
        with open(path_k,"r") as data2:
          d=json.load(data2)
          try:
            for z in d["data"]["usersByDevice"]:
              brand=z["brand"]
              count=z["count"]
              percentage=z["percentage"]
              clm1["brands"].append(brand)
              clm1["count"].append(count)
              clm1["percentage"].append(percentage)
              clm1["state"].append(i)
              clm1["year"].append(j)
              clm1["quarter"].append(int(k.strip(".json")))
          except:
            pass
agg_user=pd.DataFrame(clm1)

agg_user["state"]=agg_user["state"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
agg_user["state"]=agg_user["state"].str.replace("---"," & ")
agg_user["state"]=agg_user["state"].str.replace("-"," ")
agg_user["state"]=agg_user["state"].str.replace("Dadra & Nagar Haveli & Daman & diu","Dadra and Nagar Haveli and Daman and Diu")
agg_user["state"]=agg_user["state"].str.title()

In [13]:
agg_user

,state,year,quarter,brands,count,percentage
0,Andaman & Nicobar,2018,1,Xiaomi,1665,0.247033
1,Andaman & Nicobar,2018,1,Samsung,1445,0.214392
2,Andaman & Nicobar,2018,1,Vivo,982,0.145697
3,Andaman & Nicobar,2018,1,Oppo,501,0.074332
4,Andaman & Nicobar,2018,1,OnePlus,332,0.049258
...,...,...,...,...,...,...
6727,west bengal,2022,1,Lenovo,330017,0.015056
6728,west bengal,2022,1,Infinix,284678,0.012987
6729,west bengal,2022,1,Asus,280347,0.012790
6730,west bengal,2022,1,Apple,277752,0.012671


# map transaction

In [50]:
maptranspath="D:/data science/projects/phonepe_pulse/data/map/transaction/hover/country/india/state"
map_st_list=os.listdir(maptranspath)
clm2={"state":[],"year":[],"quarter":[],"district":[],"transaction_count":[],"transaction_amount":[]}
for i in map_st_list:
  path_i=maptranspath+"/"+i
  agg_yr=os.listdir(path_i)
  for j in agg_yr:
    path_j=path_i+"/"+j
    agg_yr_list=os.listdir(path_j)
    for k in agg_yr_list:
      path_k=path_j+"/"+k
      data=open(path_k,"r")
      d=json.load(data)
      for z in d["data"]["hoverDataList"]:
        district=z["name"]
        count=z["metric"][0]["count"]
        amount=z["metric"][0]["amount"]
        clm2["district"].append( district)
        clm2["transaction_count"].append(count)
        clm2["transaction_amount"].append(amount)
        clm2["state"].append(i)
        clm2["year"].append(j)
        clm2["quarter"].append(int(k.strip(".json")))
map_tranc=pd.DataFrame(clm2)

map_tranc["state"]=map_tranc["state"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
map_tranc["state"]=map_tranc["state"].str.replace("---"," & ")
map_tranc["state"]=map_tranc["state"].str.replace("-"," ")
map_tranc["state"]=map_tranc["state"].str.replace("Dadra & Nagar Haveli & Daman & diu","Dadra and Nagar Haveli and Daman and Diu")
map_tranc["state"]=map_tranc["state"].str.title()

In [51]:
map_tranc

,state,year,quarter,district,transaction_count,transaction_amount
0,Andaman & Nicobar,2018,1,north and middle andaman district,442,9.316631e+05
1,Andaman & Nicobar,2018,1,south andaman district,5688,1.256025e+07
2,Andaman & Nicobar,2018,1,nicobars district,528,1.139849e+06
3,Andaman & Nicobar,2018,2,north and middle andaman district,825,1.317863e+06
4,Andaman & Nicobar,2018,2,south andaman district,9395,2.394824e+07
...,...,...,...,...,...,...
14631,West Bengal,2022,4,nadia district,12690126,2.804568e+10
14632,West Bengal,2022,4,birbhum district,7617444,1.614650e+10
14633,West Bengal,2022,4,purba medinipur district,14484229,3.309949e+10
14634,West Bengal,2022,4,maldah district,12492746,2.721861e+10


# map user

In [52]:
mapuserpath="D:/data science/projects/phonepe_pulse/data/map/user/hover/country/india/state"
map_st_list1=os.listdir(mapuserpath)
clm3={"state":[],"year":[],"quarter":[],"district":[],"registeredUsers":[],"appOpens":[]}
for i in map_st_list1:
  path_i=mapuserpath+"/"+i
  agg_yr=os.listdir(path_i)
  for j in agg_yr:
    path_j=path_i+"/"+j
    agg_yr_list=os.listdir(path_j)
    for k in agg_yr_list:
      path_k=path_j+"/"+k
      data=open(path_k,"r")
      d=json.load(data)
      for z in d["data"]["hoverData"].items():
        district=z[0]
        registered_user=z[1]["registeredUsers"]
        app_opens=z[1]["appOpens"]
        clm3["state"].append(i)
        clm3["year"].append(j)
        clm3["quarter"].append(int(k.strip(".json")))
        clm3["district"].append(district)
        clm3["registeredUsers"].append(registered_user)
        clm3["appOpens"].append(app_opens)
map_user=pd.DataFrame(clm3)

map_user["state"]=map_user["state"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
map_user["state"]=map_user["state"].str.replace("---"," & ")
map_user["state"]=map_user["state"].str.replace("-"," ")
map_user["state"]=map_user["state"].str.replace("Dadra & Nagar Haveli & Daman & diu","Dadra and Nagar Haveli and Daman and Diu")
map_user["state"]=map_user["state"].str.title()

In [53]:
map_user

,state,year,quarter,district,registeredUsers,appOpens
0,Andaman & Nicobar,2018,1,north and middle andaman district,632,0
1,Andaman & Nicobar,2018,1,south andaman district,5846,0
2,Andaman & Nicobar,2018,1,nicobars district,262,0
3,Andaman & Nicobar,2018,2,north and middle andaman district,911,0
4,Andaman & Nicobar,2018,2,south andaman district,8143,0
...,...,...,...,...,...,...
14635,West Bengal,2022,4,nadia district,1359420,33853990
14636,West Bengal,2022,4,birbhum district,855236,20950662
14637,West Bengal,2022,4,purba medinipur district,1346908,38278506
14638,West Bengal,2022,4,maldah district,954892,29023743


# top transaction

In [54]:
toptranspath="D:/data science/projects/phonepe_pulse/data/top/transaction/country/india/state"
top_st_list=os.listdir(toptranspath)
clm4={"state":[],"year":[],"quarter":[],"pincode":[],"count":[],"amount":[]}
for i in top_st_list:
  path_i=toptranspath+"/"+i
  agg_yr=os.listdir(path_i)
  for j in agg_yr:
    path_j=path_i+"/"+j
    agg_yr_list=os.listdir(path_j)
    for k in agg_yr_list:
      path_k=path_j+"/"+k
      data=open(path_k,"r")
      d=json.load(data)
      for z in d["data"]["pincodes"]:
        pincode=z["entityName"]
        count=z["metric"]["count"]
        amount=z["metric"]["amount"]
        clm4["state"].append(i)
        clm4["year"].append(j)
        clm4["quarter"].append(int(k.strip(".json")))
        clm4["pincode"].append(pincode)
        clm4["count"].append(count)
        clm4["amount"].append(amount)
top_trans=pd.DataFrame(clm4)

top_trans["state"]=top_trans["state"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
top_trans["state"]=top_trans["state"].str.replace("---"," & ")
top_trans["state"]=top_trans["state"].str.replace("-"," ")
top_trans["state"]=top_trans["state"].str.replace("Dadra & Nagar Haveli & Daman & diu","Dadra and Nagar Haveli and Daman and Diu")
top_trans["state"]=top_trans["state"].str.title()

In [55]:
top_trans

,state,year,quarter,pincode,count,amount
0,Andaman & Nicobar,2018,1,744101,1622,2.769298e+06
1,Andaman & Nicobar,2018,1,744103,1223,2.238042e+06
2,Andaman & Nicobar,2018,1,744102,969,3.519060e+06
3,Andaman & Nicobar,2018,1,744105,685,1.298561e+06
4,Andaman & Nicobar,2018,1,744104,340,1.039715e+06
...,...,...,...,...,...,...
7134,West Bengal,2022,4,722101,2900058,5.748321e+09
7135,West Bengal,2022,4,700135,2471048,3.527457e+09
7136,West Bengal,2022,4,732101,2407008,5.052109e+09
7137,West Bengal,2022,4,700091,2348447,2.176640e+09


# top user

In [56]:
topuserpath="D:/data science/projects/phonepe_pulse/data/top/user/country/india/state"
top_st_list1=os.listdir(topuserpath)
clm5={"state":[],"year":[],"quarter":[],"pincode":[],"registeredUser":[]}
for i in top_st_list1:
  path_i=topuserpath+"/"+i
  agg_yr=os.listdir(path_i)
  for j in agg_yr:
    path_j=path_i+"/"+j
    agg_yr_list=os.listdir(path_j)
    for k in agg_yr_list:
      path_k=path_j+"/"+k
      data=open(path_k,"r")
      d=json.load(data)
      for z in d["data"]["pincodes"]:
        pincode=z["name"]
        registeredUser=z["registeredUsers"]
        clm5["state"].append(i)
        clm5["year"].append(j)
        clm5["quarter"].append(int(k.strip(".json")))
        clm5["pincode"].append(pincode)
        clm5["registeredUser"].append(registeredUser)
top_user=pd.DataFrame(clm5)

top_user["state"]=top_user["state"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
top_user["state"]=top_user["state"].str.replace("---"," & ")
top_user["state"]=top_user["state"].str.replace("-"," ")
top_user["state"]=top_user["state"].str.replace("Dadra & Nagar Haveli & Daman & diu","Dadra and Nagar Haveli and Daman and Diu")
top_user["state"]=top_user["state"].str.title()

In [57]:
top_user

,state,year,quarter,pincode,registeredUser
0,Andaman & Nicobar,2018,1,744103,1608
1,Andaman & Nicobar,2018,1,744101,1108
2,Andaman & Nicobar,2018,1,744105,1075
3,Andaman & Nicobar,2018,1,744102,1006
4,Andaman & Nicobar,2018,1,744104,272
...,...,...,...,...,...
7135,West Bengal,2022,4,700015,108457
7136,West Bengal,2022,4,742304,105471
7137,West Bengal,2022,4,721101,105279
7138,West Bengal,2022,4,700091,102363


#  sql Aggregate_Transaction

In [66]:
cursor.execute('''create table if not exists Aggregate_Transaction(state varchar(50),
           year int, 
           quarter int, 
           transaction_type varchar(50),
           transaction_count bigint,
           transaction_amount bigint)'''
           )
conn.commit()
for _, row in agg_trans.iterrows():
            insert_query = '''
                INSERT INTO Aggregate_Transaction (state, year, quarter, transaction_type, transaction_count, transaction_amount)
                VALUES (%s, %s, %s, %s, %s, %s)

            '''
            values = (
                row['state'],
                row['year'],
                row['quarter'],
                row['transaction_type'],
                row['transaction_count'],
                row['transaction_amount']
            )
            cursor.execute(insert_query,values)
conn.commit()


#  sql Aggregate_user

In [67]:
cursor.execute('''create table if not exists Aggregate_user(state varchar(50),
           year int, 
           quarter int, 
           brands varchar(50),
           count bigint,
           percentage float)'''
           )
conn.commit()
for _, row in agg_user.iterrows():
            insert_query = '''
                INSERT INTO Aggregate_user (state, year, quarter, brands, count, percentage)
                VALUES (%s, %s, %s, %s, %s, %s)

            '''
            values = (
                row['state'],
                row['year'],
                row['quarter'],
                row['brands'],
                row['count'],
                row['percentage']
            )
            cursor.execute(insert_query,values)
conn.commit()


# sql Map_Transaction

In [68]:
cursor.execute('''create table if not exists Map_Transaction(state varchar(50),
           year int, 
           quarter int, 
           districts varchar(50),
           transaction_count bigint,
           transaction_amount bigint)'''
           )
conn.commit()
for _, row in map_tranc.iterrows():
            insert_query = '''
                INSERT INTO Map_Transaction (state, year, quarter, districts, transaction_count, transaction_amount)
                VALUES (%s, %s, %s, %s, %s, %s)

            '''
            values = (
                row['state'],
                row['year'],
                row['quarter'],
                row['district'],
                row['transaction_count'],
                row['transaction_amount']
            )
            cursor.execute(insert_query,values)
conn.commit()


# sql Map_user

In [69]:
cursor.execute('''create table if not exists Map_user(state varchar(50),
           year int, 
           quarter int, 
           districts varchar(50),
           registeredUsers bigint,
           appOpens bigint)'''
           )
conn.commit()
for _, row in map_user.iterrows():
            insert_query = '''
                INSERT INTO Map_user (state, year, quarter, districts,registeredUsers,appOpens)
                VALUES (%s, %s, %s, %s, %s, %s)

            '''
            values = (
                row['state'],
                row['year'],
                row['quarter'],
                row['district'],
                row['registeredUsers'],
                row['appOpens']
            )
            cursor.execute(insert_query,values)
conn.commit()


# sql Top_Transaction

In [70]:
cursor.execute('''create table if not exists Top_Transaction(state varchar(50),
           year int, 
           quarter int, 
           pincode int,
           transaction_count bigint,
           transaction_amount bigint)'''
           )
conn.commit()
for _, row in top_trans.iterrows():
            insert_query = '''
                INSERT INTO Top_Transaction (state, year, quarter, pincode, transaction_count, transaction_amount)
                VALUES (%s, %s, %s, %s, %s, %s)

            '''
            values = (
                row['state'],
                row['year'],
                row['quarter'],
                row['pincode'],
                row['count'],
                row['amount']
            )
            cursor.execute(insert_query,values)
conn.commit()


#  sql top user

In [71]:
cursor.execute('''create table if not exists Top_user(state varchar(50),
           year int, 
           quarter int, 
           pincode int,
           registeredUser bigint)'''
           )
conn.commit()
for _, row in top_user.iterrows():
            insert_query = '''
                INSERT INTO Top_user (state, year, quarter, pincode, registeredUser)
                VALUES (%s, %s, %s, %s, %s)

            '''
            values = (
                row['state'],
                row['year'],
                row['quarter'],
                row['pincode'],
                row['registeredUser'],
            )
            cursor.execute(insert_query,values)
conn.commit()
